In [2]:
from data_prep_utils import *
from plot_prep_utils import *
import pandas as pd

dfe = get_eigenphi_march_blocks_with_to_and_from()
dft = get_titan_march_blocks_with_to_and_from()

In [5]:
print(len(dfe["to_address"].unique()))
print(len(dfe["from_address"].unique()))
print(len(dft["to_address"].unique()))
print(len(dft["from_address"].unique()))



5842
7802
1682447
2760243


In [8]:


def get_top_rewards(df, address_type, output_file):
    """
    Processes rewards data to return the top 50 addresses and optionally saves it to a CSV file.
    
    Args:
    df (pd.DataFrame): DataFrame containing the transaction data.
    address_type (str): 'to_address' or 'from_address' for reward aggregation.
    
    Returns:
    pd.DataFrame: DataFrame holding the top 50 addresses sorted by rewards.
    """
    if address_type not in ['to_address', 'from_address']:
        raise ValueError("address_type must be 'to_address' or 'from_address'")

    subtotal_df = df.groupby(address_type)["builder_reward"].sum().reset_index()
    subtotal_df["builder_reward"] = subtotal_df["builder_reward"].astype(float)
    sorted_df = subtotal_df.sort_values("builder_reward", ascending=False).head(50)

    if output_file:
        sorted_df.to_csv(output_file, index=False)
        print(f"DataFrame exported to {output_file}")

    return sorted_df


In [9]:
top50_subtotal_from_dft = get_top_rewards(dft, 'from_address', 'top_50_titan_rewards_from.csv')
top50_subtotal_to_dft = get_top_rewards(dft, 'to_address', 'top_50_titan_rewards_to.csv')
top50_subtotal_from_dfe = get_top_rewards(dfe, 'from_address', 'top_50_eigenphi_rewards_from.csv')   
top50_subtotal_to_dfe = get_top_rewards(dfe, 'to_address', 'top_50_eigenphi_rewards_to.csv')


DataFrame exported to top_50_titan_rewards_from.csv
DataFrame exported to top_50_titan_rewards_to.csv
DataFrame exported to top_50_eigenphi_rewards_from.csv
DataFrame exported to top_50_eigenphi_rewards_to.csv


In [10]:
import pandas as pd

def find_common_addresses(df1, df2, address_column, output_file):
    """
    Finds common addresses between two datasets and saves the result to a CSV file.

    Args:
        df1 (pd.DataFrame): First dataset containing addresses.
        df2 (pd.DataFrame): Second dataset containing addresses.
        address_column (str): The name of the column to merge on ('from_address' or 'to_address').
        output_file (str): The filename for the output CSV.
    """
    # Merging on specified address column
    common_addresses = pd.merge(df1, df2, on=address_column)
    print(f"Common {address_column} between datasets:")
    # print(common_addresses)

    # Save to CSV
    common_addresses.to_csv(output_file, index=False)
    # print(f"DataFrame exported to {output_file}")

# Assuming subtotal_from_dft, subtotal_from_dfe, subtotal_to_dft, and subtotal_to_dfe are already defined:

# Find common from_addresses between EigenPhi and Titan
find_common_addresses(top50_subtotal_from_dft, top50_subtotal_from_dfe, 'from_address', 'common_from_addresses.csv')

# Find common to_addresses between EigenPhi and Titan
find_common_addresses(top50_subtotal_to_dft, top50_subtotal_to_dfe, 'to_address', 'common_to_addresses.csv')


Common from_address between datasets:
Common to_address between datasets:


In [16]:
# find uncommon addresses between EigenPhi and Titan
import pandas as pd

def find_uncommon_addresses(df1, df2, address_column, eigenphi_output_file, titan_output_file):
    """
    Finds uncommon addresses between two datasets and saves the result to a CSV file.

    Args:
        df1 (pd.DataFrame): First dataset containing addresses.
        df2 (pd.DataFrame): Second dataset containing addresses.
        address_column (str): The name of the column to merge on ('from_address' or 'to_address').
        output_file (str): The filename for the output CSV.
    """
    # Assuming subtotal_from_dft and subtotal_from_dfe are already defined DataFrame from previous steps:
    # Perform an outer join merge
    all_addresses = pd.merge(df1, df2, on=address_column, how='outer', indicator=True)
    
    # Filter results to get non-overlapping addresses
    uncommon_addresses = all_addresses[all_addresses['_merge'] != 'both']
    
    
    # pd.merge adds a column '_merge' that indicates from which DataFrame the rows come from:
    # "left_only" for EigenPhi, "right_only" for Titan, and "both" for common.

    # Optionally, split into two DataFrames depending on source
    uncommon_eigenphi = uncommon_addresses[uncommon_addresses['_merge'] == 'left_only']
    uncommon_titan = uncommon_addresses[uncommon_addresses['_merge'] == 'right_only']

    uncommon_eigenphi[[address_column]].to_csv(eigenphi_output_file, index=False)
    uncommon_titan[[address_column]].to_csv(titan_output_file, index=False)
    
    
# Find uncommon from_addresses between EigenPhi and Titan
find_uncommon_addresses(top50_subtotal_from_dfe, top50_subtotal_from_dft, 'from_address', 'uncommon_from_addresses_eigenphi.csv', 'uncommon_from_addresses_titan.csv')


# Find uncommon to_addresses between EigenPhi and Titan
find_uncommon_addresses(top50_subtotal_to_dfe, top50_subtotal_to_dft, 'to_address', 'uncommon_to_addresses_eigenphi.csv', 'uncommon_to_addresses_titan.csv')


In [25]:
address_list = ["0x3328f7f4a1d1c57c35df56bbf0c9dcafca309c49",
"0x80a64c6d7f12c47b7c66c5b4e20e72bc1fcd5d9e",
"0xdac17f958d2ee523a2206206994597c13d831ec7",
"0x2b8bc3f432f5a95aadd0593d39e8bf8a0174b7a4",
"0xbeb5fd030ffb0fbc95d68113c1c796eff65526d7",
"0xf1b0475ab021f37b971013b4222e510a8679bedf",
"0xf3de3c0d654fda23dad170f0f320a92172509127",
"0x00000023c10000eecb940000b914cdfd76cc83d1",
"0xa5c7fe26c0a339fb449ca8c9d565dcf44472ac86",
"0x99b1817acb40e76c309e26b2face9da9eff55317",
"0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
"0xa4a343f900b1f0afc140af985f307b53629b1bca",
"0x738e79fbc9010521763944ddf13aad7f61502221",
"0xe5bba2bde7d4192d4b986e9e87a39f0c0dadeb38",
"0x4736b02db015dcd1a57a69c889d073b100000000",
"0x360e051a25ca6decd2f0e91ea4c179a96c0e565e",
"0x9696a5c3eb572de180aa7f76e39c0f4418a34af1",
"0x6719c6ebf80d6499ca9ce170cda72beb3f1d1a54",
"0xdc3bfcb3521cd02af66d394f817a0e6fe62ded95",
"0x767c8bb1574bee5d4fe35e27e0003c89d43c5121",
"0x4e4bc98940a3ac33828377e41aef91c611367046",
"0xe8cfad4c75a5e1caf939fd80afcf837dde340a69",
"0xbb8f69fc5696e66d8b22f981f4833f3354d52e62",
"0x73b2416e1cbafbf6112abfdcf10991bd0705f715",
"0xb0000000aa4f00af1200c8b2befb6300853f0069",
"0xa0425d71cb1d6fb80e65a5361a04096e0672de03",
"0x5a9b39241d443038c22da6028fffc48bfd97d4d2",]
contract_df = dft[dft["to_address"].isin(address_list)]

contract_df = contract_df.groupby("to_address").apply(lambda x: x.nlargest(50, "builder_reward")).reset_index(drop=True)

contract_df.to_csv("top_50_titan_rewards_for_contract_list.csv", index=False)



/var/folders/mf/lxpn1ltx7t97r1v1k92vly1w0000gn/T/ipykernel_72420/451699516.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  contract_df = contract_df.groupby("to_address").apply(lambda x: x.nlargest(50, "builder_reward")).reset_index(drop=True)
